
**This file prepares the data we will need for machine learning.**

It opens specified Wireshark capture files and cleans it up in an appropriate manner.

In [1]:
# Imports
from scapy.all import *
import pandas as pd
import ipaddress
import csv
from helper import processFile, ip_to_numeric

Here, the capture files are grabbed from the appropriate directories and labled

In [2]:
# From captureFiles directory
files = []

dir_list = os.listdir("./captureFilesAttack/")
for each in dir_list:
    files.append(("captureFilesAttack/"+each,"attack"))

dir_list = os.listdir("./captureFilesNormal/")
for each in dir_list:
    files.append(("captureFilesNormal/"+each,"normal"))

files

[('captureFilesAttack/attack.pcap', 'attack'),
 ('captureFilesNormal/http-cnn2011.pcapng', 'normal'),
 ('captureFilesNormal/http-espn2012.pcapng', 'normal'),
 ('captureFilesNormal/http-msnbc.pcapng', 'normal'),
 ('captureFilesNormal/http-wiresharkdownload.pcapng', 'normal'),
 ('captureFilesNormal/ssl_saintcon.pcapng', 'normal')]

Now, actually doing processing on our files

In [3]:
# Making sure the file is clean
file_to_delete = open("normalizedData/normalizedLearn.csv",'w')
file_to_delete.write("source,destination,source_port,destination_port,protocol,length,flags,ttl,window_size,seq_number,ack_number,time_delta,label\n")
file_to_delete.close()

In [4]:
# Actually processing the files. This one takes a while.
# Third argument is the path to where to store the files 
for pcap_file in files:
    processFile(pcap_file[0], pcap_file[1], path="normalizedData/normalizedLearn.csv", type="a")

file = pd.read_csv('normalizedData/normalizedLearn.csv')
file.sample(n=4)

,source,destination,source_port,destination_port,protocol,length,flags,ttl,window_size,seq_number,ack_number,time_delta,label
30089,24.6.173.220,69.4.231.52,25921,80,6,40,16,128,65335,1092584662,305509961,0.000235,normal
16726,192.221.123.254,24.6.173.220,80,65473,6,52,18,58,5840,601273155,3062714797,0.000004,normal
7767,100.36.203.15,10.10.10.10,0,0,1,1154,0,104,0,0,0,0.000003,attack
18016,65.54.87.113,24.6.173.220,80,65520,6,1500,16,55,6296,3401748432,4285975399,0.001056,normal


In [5]:
file.sample(n=8)

,source,destination,source_port,destination_port,protocol,length,flags,ttl,window_size,seq_number,ack_number,time_delta,label
7947,107.3.147.107,10.10.10.10,0,0,1,126,0,135,0,0,0,0.000004,attack
14694,184.84.222.88,24.6.173.220,80,19996,6,1500,16,59,274,4150886272,4071427940,0.000857,normal
19033,69.4.231.52,24.6.173.220,80,25921,6,1500,16,51,14,294297161,1092584662,0.000757,normal
23123,69.4.231.52,24.6.173.220,80,25921,6,1500,16,51,14,298447941,1092584662,0.000904,normal
6538,17.59.231.158,10.10.10.10,0,0,1,243,0,86,0,0,0,0.000003,attack
10439,96.17.148.122,24.6.173.220,80,62111,6,1062,24,59,216,262169992,402779957,0.000003,normal
2325,95.0.34.241,10.10.10.10,0,0,1,286,0,63,0,0,0,0.000002,attack
16828,24.6.173.220,192.221.123.254,65487,80,6,548,24,128,16425,1286045354,680828878,0.000005,normal


**Preparations**

Since Machine Learning Models cannot work on categorical variables in the form of strings, we have to clean the data in away appropriate for ML algorithms.

So, we have to convert IP addresses from dotted format to decimal

In [6]:
update_file = file

update_file['source'] = update_file['source'].apply(ip_to_numeric)
update_file['destination'] = update_file['destination'].apply(ip_to_numeric)

update_file.to_csv('normalizedData/normalizedLearn.csv', index = False)

update_file.sample(n=5)

,source,destination,source_port,destination_port,protocol,length,flags,ttl,window_size,seq_number,ack_number,time_delta,label
26650,403090908,1157949236,25921,80,6,40,16,128,65335,1092584662,302033701,0.000181,normal
2972,3325704841,168430090,0,0,1,374,0,57,0,0,0,0.000003,attack
25198,1157949236,403090908,80,25921,6,1500,16,51,14,300551801,1092584662,0.000823,normal
1583,2517782712,168430090,0,0,1,1236,0,74,0,0,0,0.000002,attack
5972,373240384,168430090,0,0,1,456,0,51,0,0,0,0.000004,attack


Perfect! Now onto actual machine learning in MLmodels.ipnb